# Chapter 3: Models
In this chapter, we explain the ideas behind model and agent interfaces and the example model/agent implementations provided by `ldmunit`.

## Capabilities
Similar to `sciunit`, `ldmunit` test classes require models to have certain capabilities. For example, `InteractiveTest` requires the models to implement the requirements specified by `ldmunit.capabilities.Interactive` capability. In addition to test classes, the score class used for testing can also require capabilities. If you want to use `ldmunit.scores.AICScore`, then the model should also have `ldmunit.capabilities.PredictsLogpdf` and `ldmunit.capabilities.ReturnsNumParams` capabilities. We can see these requirements as follows:

In [2]:
from ldmunit.testing import InteractiveTest
from ldmunit.scores import AICScore
print(InteractiveTest.required_capabilities)
print(AICScore.required_capabilities)

(<class 'ldmunit.capabilities.Interactive'>,)
(<class 'ldmunit.capabilities.PredictsLogpdf'>, <class 'ldmunit.capabilities.ReturnsNumParams'>)


## Model Interface
In `ldmunit` terms, a model represents a way of predicting the actions of some agent and fitting parameters to data:

In [4]:
from ldmunit.models import LDMModel
print('LDMModel Documentation')
print('----------------------')
print(LDMModel.__doc__)

LDMModel Documentation
----------------------

    Base class for LDMUnit models.

    In `ldmunit`, a model is a way of representing a continuum of some type of an agent and corresponding possible parameters. Models are
    expected to provide fitting and action prediction functionalities, while leaving the tasks of acting on environments and
    updating hidden state variables to agents.
    


## Agent Interface
An agent represents a way of interacting with an environment through `act` and `update` methods:

In [5]:
from ldmunit.models import LDMAgent
print('LDMAgent Documentation')
print('----------------------')
print(LDMAgent.__doc__)

LDMAgent Documentation
----------------------

    Base class for LDMUnit agents.

    In `ldmunit`, an agent is a way of interacting with environments through `act` and `update` methods while possibly
    storing some hidden state.
    


## Example Agents and Models
In this section, we go over example model and agent implementations provided by ldmunit:

In [10]:
# import Rescorla-Wagner Choice Kernel model
from ldmunit.models.decision_making import RWCKModel, RWCKAgent

If we look at the superclasses of `RWCKAgent`, we see that it inherits from

In [12]:
RWCKAgent.__bases__

(ldmunit.models.base.LDMAgent,
 ldmunit.capabilities.ProducesPolicy,
 ldmunit.capabilities.DiscreteAction,
 ldmunit.capabilities.DiscreteObservation)

Here, the last two define the action and observation space this agent can interact with. This is important for methods that require environment and agent interaction such as `ldmunit.simulation.simulate` function. Then, we see that `RWCKAgent` inherits from `ProducesPolicy` capability. This means that `RWCKAgent` provides `eval_policy` method that returns a probability distribution over the action space. Although this is not directly required for interacting with environments, it allows a very straightforward model implementation. In `ldmunit`, this functionality is in `ldmunit.models.policy_model.PolicyModel` class.

In [13]:
from ldmunit.models.policy_model import PolicyModel
print(PolicyModel.__doc__)


    PolicyModel provides a model implementation that can be created from agents satisfying :class:`ldmunit.capabilities.ProducesPolicy`
    capability.

    If you already have an agent implementation that can provide a probability distribution over the action space (`eval_policy` method), you
    can create a model of that agent that uses `eval_policy` to make predictions and to fit model parameters (using maximum likelihood) by simply
    deriving from this class. For examples of this, refer to decision making or associative learning model implementations
    provided by `ldmunit`.
    


 Any model that subclass from `PolicyModel` automatically gets `fit` and `predict` method implementations by using the `eval_policy` method of the underlying agent. `RWCKModel` is an example of such implementation; it subclasses from `PolicyModel` and uses an `RWCKAgent` for the underlying update equations and probability distribution generation.

In [17]:
RWCKModel.__bases__

(ldmunit.models.policy_model.PolicyModel,
 ldmunit.capabilities.DiscreteAction,
 ldmunit.capabilities.DiscreteObservation)

## Implementing Your Own Agents or Models
You can implement models by taking any of the examples as a starting point. For interactive tests, satisfying `ldmunit.capabilities.Interactive` is a requirement. Apart from this, models can continuously satisfy new capabilities in order to take new tests.